In [149]:
import pickle
import numpy as np
from keras.models import Sequential
import keras.layers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [150]:
np.random.seed(0)
train, valid, test = pickle.load(open('dataset.pickle', 'rb'))
train = train.fillna(0)
test = test.fillna(0)

In [151]:
x_train = train[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_train = train.iloc[:,-8:]
x_test = test[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_test = np.asarray(test.iloc[:,-8:])
y_test_decode = []
for i in range(y_test.shape[0]):
    y_test_decode.insert(len(y_test_decode),np.argmax(y_test[i,:]))

In [152]:
scaler = MinMaxScaler() #scale features between 0 and 1
x_train = scaler.fit_transform(x_train)
x_train = np.asarray(x_train).reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = scaler.fit_transform(x_test)
x_test = np.asarray(x_test).reshape(x_test.shape[0], 1, x_test.shape[1])

In [153]:
model = Sequential()
model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2]), bias_regularizer=L1L2(l1=0.01, l2=0.01)))
#model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(0.2, noise_shape=None, seed=None))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [154]:
history = model.fit([x_train], batch_size=20, y=y_train, verbose=1, validation_split=0.25, 
          shuffle=True, epochs=5)

Train on 227768 samples, validate on 75923 samples
Epoch 1/5
227768/227768 [==============================] - 183s 803us/step - loss: 1.4091 - acc: 0.6398 - val_loss: 1.3515 - val_acc: 0.5391
Epoch 2/5
227768/227768 [==============================] - 172s 754us/step - loss: 0.7824 - acc: 0.7182 - val_loss: 1.6044 - val_acc: 0.4601
Epoch 3/5
227768/227768 [==============================] - 172s 755us/step - loss: 0.6919 - acc: 0.7531 - val_loss: 1.5209 - val_acc: 0.5302
Epoch 4/5
227768/227768 [==============================] - 156s 687us/step - loss: 0.6458 - acc: 0.7705 - val_loss: 1.5656 - val_acc: 0.5086
Epoch 5/5
227768/227768 [==============================] - 156s 687us/step - loss: 0.6101 - acc: 0.7832 - val_loss: 1.7244 - val_acc: 0.4705


In [155]:
predicted = model.predict(x_test)
predicted = np.argmax(predicted, axis=1)
print(accuracy_score(y_test_decode, predicted))

0.4996832344640903


In [165]:
a= [0,1,1]
print(len(y_test_decode))
print(np.count_nonzero(y_test_decode))

34726
26884
